# Simple ideas of fusing modalities

- There are many ways to fuse modalities, in this we will start with a state-of-the-art approach where fusion happens in the intermediate layers
- An alternative to this - idea is feature/sensor concatenation, which is shown to be inferior by many research papers

# Getting Started
- For this example, we will use two inertial sensors: Acc and Gyro
- This fusion is very easy to do, as they both come from same device, and are often captured at the same sampling intervals
- In real scenario's it is often challenging, due to multiple reasons outlined in my past research
- Paper: Time Awareness in Deep Learning-based Multimodal Fusion across Smartphone Platforms. IoTDI, 2020

# Individual Modality: Acc Accuracy: 84%
# Individual Modality Gyro Accuracy: 76%

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.utils import to_categorical

from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot

2022-03-09 13:03:40.122883: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [ ]:
import tensorflow as tf
tf.config.experimental.set_memory_growth(True)

# Data loading

In [3]:
def load_file(filepath):
    dataframe = read_csv(filepath, header=None, delim_whitespace=True)
    return dataframe.values
 
def load_group(filenames, prefix=''):
    loaded = list()
    for name in filenames:
        data = load_file(prefix + name)
        loaded.append(data)
        
    loaded = dstack(loaded)
    return loaded

In [4]:
# load train or test
def load_dataset_group(group, prefix=''):
    filepath = prefix + group + '/Inertial Signals/'
    # load all 9 files as a single array
    filenames = list()
    # total acceleration
    filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
    # body acceleration
    filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
    # body gyroscope
    filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
    # load input data
    X = load_group(filenames, filepath)
    # load class output
    y = load_file(prefix + group + '/y_'+group+'.txt')
    return X, y

In [5]:
# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
    # load all train
    trainX, trainy = load_dataset_group('train', prefix + 'datasets/UCI-HAR-Dataset/')
    
    # load all test
    testX, testy = load_dataset_group('test', prefix + 'datasets/UCI-HAR-Dataset/')
    
    # zero-offset class values
    trainy = trainy - 1
    testy = testy - 1
    # one hot encode y
    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    print(trainX.shape, trainy.shape, testX.shape, testy.shape)
    return trainX, trainy, testX, testy

In [6]:
# Load the data
trainX, trainy, testX, testy = load_dataset()

(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)


In [7]:
trainX_acc = trainX[:,:,:3]
print(trainX_acc.shape)
testX_acc = testX[:,:,:3]
print(testX_acc.shape)

trainX_gyro = trainX[:,:,6:]
print(trainX_gyro.shape)
testX_gyro = testX[:,:,6:]
print(testX_gyro.shape)

(7352, 128, 3)
(2947, 128, 3)
(7352, 128, 3)
(2947, 128, 3)


# Individual modality model

In [8]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, MaxPooling1D, Flatten
from tcn import TCN

In [9]:
def get_model_tcn(n_timesteps, n_features, nb_filters, n_outputs):
    model = Sequential()
    model.add(TCN(input_shape=(n_timesteps, n_features),
        nb_filters=nb_filters,
        kernel_size=3,
        nb_stacks=3,
        use_skip_connections=False,
        use_batch_norm=False,
        use_weight_norm=False,
        use_layer_norm=False))
    
    model.add(Flatten())
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
   
    return model

In [10]:
# verbose, epochs, batch_size = 1, 50, 128
# n_timesteps, n_features, n_outputs = trainX_acc.shape[1], trainX_acc.shape[2], trainy.shape[1]

# model_acc = get_model_tcn(n_timesteps, n_features, 16, n_outputs)

2022-03-09 13:03:44.138113: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-03-09 13:03:44.139642: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-03-09 13:03:44.196046: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:02:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.683GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 451.17GiB/s
2022-03-09 13:03:44.196076: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-03-09 13:03:44.198036: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2022-03-09 13:03:44.198080: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.

In [11]:
model_acc.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
tcn (TCN)                    (None, 16)                27664     
_________________________________________________________________
flatten (Flatten)            (None, 16)                0         
_________________________________________________________________
dropout (Dropout)            (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 64)                1088      
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 390       
Total params: 29,142
Trainable params: 29,142
Non-trainable params: 0
_________________________________________________________________


In [12]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)

In [13]:
model_acc.fit(trainX_acc, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)

Epoch 1/50


/home/nesl/anaconda3/envs/tf25/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py:3503: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  warnings.warn(
2022-03-09 13:03:45.743624: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-03-09 13:03:45.764102: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3497870000 Hz
2022-03-09 13:03:45.786580: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8
2022-03-09 13:03:46.559665: W tensorflow/stream_executor/gpu/asm_compiler.cc:63] Running ptxas --version returned 256
2022-03-09 13:03:46.621900: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output:

58/58 [==============================] - 38s 411ms/step - loss: 178.8223 - accuracy: 0.2117
Epoch 2/50
58/58 [==============================] - 10s 172ms/step - loss: 2.5397 - accuracy: 0.3882
Epoch 3/50
58/58 [==============================] - 10s 173ms/step - loss: 1.6706 - accuracy: 0.5041
Epoch 4/50
58/58 [==============================] - 10s 179ms/step - loss: 1.2618 - accuracy: 0.5817
Epoch 5/50
58/58 [==============================] - 11s 184ms/step - loss: 1.0368 - accuracy: 0.6302
Epoch 6/50
58/58 [==============================] - 10s 173ms/step - loss: 0.9058 - accuracy: 0.6548
Epoch 7/50
58/58 [==============================] - 10s 176ms/step - loss: 0.8167 - accuracy: 0.6850
Epoch 8/50
58/58 [==============================] - 10s 173ms/step - loss: 0.7643 - accuracy: 0.6976
Epoch 9/50
58/58 [==============================] - 10s 176ms/step - loss: 0.6577 - accuracy: 0.7427
Epoch 10/50
58/58 [==============================] - 10s 177ms/step - loss: 0.6363 - accuracy: 0.752

In [15]:
# evaluate model
_, accuracy = model_acc.evaluate(testX_acc, testy, batch_size=batch_size, verbose=0)

print('Test accuracy of model is:', accuracy)

Test accuracy of model is: 0.8401764631271362


In [16]:
verbose, epochs, batch_size = 1, 50, 128
n_timesteps, n_features, n_outputs = trainX_gyro.shape[1], trainX_gyro.shape[2], trainy.shape[1]

model_gyro = get_model_tcn(n_timesteps, n_features, 16, n_outputs)

In [17]:
model_gyro.fit(trainX_gyro, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)

Epoch 1/50
58/58 [==============================] - 10s 176ms/step - loss: 43.7784 - accuracy: 0.1805
Epoch 2/50
58/58 [==============================] - 10s 177ms/step - loss: 2.3417 - accuracy: 0.3057
Epoch 3/50
58/58 [==============================] - 10s 175ms/step - loss: 1.7606 - accuracy: 0.3309
Epoch 4/50
58/58 [==============================] - 10s 178ms/step - loss: 1.5119 - accuracy: 0.3250
Epoch 5/50
58/58 [==============================] - 10s 175ms/step - loss: 1.3982 - accuracy: 0.3516
Epoch 6/50
58/58 [==============================] - 10s 174ms/step - loss: 1.3043 - accuracy: 0.3656
Epoch 7/50
58/58 [==============================] - 10s 176ms/step - loss: 1.2544 - accuracy: 0.3691
Epoch 8/50
58/58 [==============================] - 10s 173ms/step - loss: 1.2221 - accuracy: 0.3804
Epoch 9/50
58/58 [==============================] - 10s 174ms/step - loss: 1.1812 - accuracy: 0.3865
Epoch 10/50
58/58 [==============================] - 10s 174ms/step - loss: 1.1553 - accur

In [18]:
# evaluate model
_, accuracy = model_gyro.evaluate(testX_gyro, testy, batch_size=batch_size, verbose=0)

print('Test accuracy of model is:', accuracy)

Test accuracy of model is: 0.7631489634513855
